In [40]:
import sympy as sym
import dill
from os import path
import numpy as np
import os
import pickle
import gzip

from data_utils import generate_data


# Test

In [15]:
a, b = sym.symbols("a, b")
expr = sym.sin(a) + sym.cos(b)
f1 = sym.lambdify((a, b), expr, modules="numpy")

In [4]:
dill.settings['recurse'] = True
dill.dump(f1, open("myfile", "wb"))

In [7]:
f2 = dill.load(open("myfile", "rb"))

In [8]:
f2(3,2)

-0.2750268284872752

In [11]:
f1

<function _lambdifygenerated(a, b)>

# Extract equation

In [13]:
save_path = 'results/1'

In [21]:
result_exp = dill.load(open(path.join(save_path, 'sym_y' + '0'), "rb"))

In [24]:
a = result_exp.split("\n")
expr = sym.sympify(a[0])
var = sym.sympify(a[1])

In [31]:
expr

-0.334780060566978*x_3*x_4 + 0.334446882513782*sin(3.1425929069519*x_1) + 0.784025678409106*sin(0.432340711355209*x_2) + 0.332726830789688*cos(6.2939133644104*x_2 - 1.17655837535858) - 0.000385077821833126

In [30]:
l_expr = sym.lambdify(var, expr, modules="numpy")
l_expr(1,1,2,3)

-1.5498051237696138

# Generate input files: train and test

In [37]:
data_gen_params = {'file_name': 'F1data',  # file name for the generated data file, will be created in data/file_name
                   'fn_to_learn': 'F1',  # python function to learn, should be defined in data_utils
                   'train_val_examples': 10000,  # total number of examples for training and validation
                   'train_val_bounds': (-1.0, 1.0),  # domain boundaries for validation and training normal epochs
                   'test_examples': 5000,  # number of test examples, if set to None no test_data file is created
                   'test_bounds': (-2.0, 2.0),  # domain boundaries for test data
                   'noise': 0.01,
                   'seed': None
                   }

In [38]:
def F1(x1, x2, x3, x4):
    """Requires 1 hidden layer."""
    y0 = (np.sin(np.pi * x1) + np.sin(2 * np.pi * x2 + np.pi / 8.0) + x2 - x3 * x4) / 3.0
    return y0,

In [41]:
fn_to_learn = F1
if not os.path.exists('data'):
    os.mkdir('data')
train_val_set = generate_data(fn=fn_to_learn, num_examples=data_gen_params["train_val_examples"], bounds=data_gen_params["train_val_bounds"], noise=data_gen_params["noise"],
                                seed=data_gen_params["seed"])
train_val_data_file = os.path.join('data', data_gen_params["file_name"] + '_train_val')
pickle.dump(train_val_set, gzip.open(train_val_data_file, "wb"))
print('Successfully created train/val data file in %s.' % train_val_data_file)

Successfully created train/val data file in data/F1data_train_val.


In [44]:
test_set = generate_data(fn=fn_to_learn, num_examples=data_gen_params["test_examples"], bounds=data_gen_params["test_bounds"], noise=data_gen_params["noise"],
                                seed=data_gen_params["seed"])
test_data_file = os.path.join('data', data_gen_params["file_name"] + '_test')
pickle.dump(test_set, gzip.open(test_data_file, "wb"))
print('Successfully created test data file in %s.' % test_data_file)

Successfully created test data file in data/F1data_test.


In [47]:
!python3 train.py '{"train_val_file": "data/F1data_train_val", "test_file": "data/F1data_test"}'

/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: 

In [46]:
!python3 train.py '{"train_val_file": "data/F1data_train_val", "test_file": "data/F1data_test", "num_h_layers": 4}'

/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kxu9/NN/env37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: 